In [307]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

data = pd.read_csv("../data/WorkoutExport.csv")

# Read and Reshape Data

I'm not importing this data from a FitBod export, which tracks each set as a row, uses multipliers, doesn't contain calculated volumes, and does requires some unit conversions.

In [308]:
# data["Date"] = pd.to_datetime(data["Date"])
data.dtypes

Date            object
Exercise        object
Reps             int64
Weight(kg)     float64
Duration(s)    float64
Distance(m)    float64
Incline        float64
Resistance     float64
isWarmup          bool
Note           float64
multiplier     float64
dtype: object

In [309]:
data.info()
"""
    These don't provide much value as I'm calculating total volume (including warmups),
    and resistance/notes are not features I utilize in the FitBod application
"""
data.drop(["Note", "isWarmup", "Resistance"], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1176 entries, 0 to 1175
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1176 non-null   object 
 1   Exercise     1176 non-null   object 
 2   Reps         1176 non-null   int64  
 3   Weight(kg)   1176 non-null   float64
 4   Duration(s)  1176 non-null   float64
 5   Distance(m)  1176 non-null   float64
 6   Incline      1176 non-null   float64
 7   Resistance   1176 non-null   float64
 8   isWarmup     1176 non-null   bool   
 9   Note         0 non-null      float64
 10  multiplier   1176 non-null   float64
dtypes: bool(1), float64(7), int64(1), object(2)
memory usage: 93.1+ KB


In [310]:
data.head(80)

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,multiplier
0,2022-02-21 18:19:51 +0000,Walking,0,0.000000,1800.358674,2867.220102,0.0,0.0
1,2022-02-21 17:29:51 +0000,Dumbbell Bench Press,12,20.411681,0.000000,0.000000,0.0,2.0
2,2022-02-21 17:29:51 +0000,Dumbbell Bench Press,12,20.411681,0.000000,0.000000,0.0,2.0
3,2022-02-21 17:29:51 +0000,Dumbbell Bench Press,12,20.411681,0.000000,0.000000,0.0,2.0
4,2022-02-21 17:29:51 +0000,Dumbbell Bench Press,10,20.411681,0.000000,0.000000,0.0,2.0
...,...,...,...,...,...,...,...,...
75,2022-02-15 22:00:39 +0000,Dumbbell Row,10,18.143716,0.000000,0.000000,0.0,2.0
76,2022-02-15 22:00:39 +0000,Dumbbell Row,10,18.143716,0.000000,0.000000,0.0,2.0
77,2022-02-15 22:00:39 +0000,Dumbbell Fly,10,9.071858,0.000000,0.000000,0.0,2.0
78,2022-02-15 22:00:39 +0000,Dumbbell Fly,10,9.071858,0.000000,0.000000,0.0,2.0


Now that we have a clean dataframe, let's convert weight to pounds, calculate a volume for each set (row), and set a "Sets" column to 1 across the board for summing later.

In [311]:
kg_to_lb_conversion = 2.20462
data["Weight(lb)"] = data["Weight(kg)"] * kg_to_lb_conversion
data["Volume(lb)"] = data["Weight(lb)"] * data["multiplier"] * data["Reps"]
data["Sets"] = 1

this_week = data.loc[(data["Date"] >= "2022-02-18")]
# this_week.drop(["Weight(kg)"], axis=1, inplace=True)

this_week.tail()

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,multiplier,Weight(lb),Volume(lb),Sets
62,2022-02-18 19:39:32 +0000,Single Leg Kickback,10,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,1
63,2022-02-18 19:39:32 +0000,Single Leg Kickback,10,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,1
64,2022-02-18 19:39:32 +0000,Single Leg Kickback,10,0.000000,0.0,0.0,0.0,2.0,0.0,0.0,1
65,2022-02-18 19:39:32 +0000,Dumbbell Lunge,6,18.143716,0.0,0.0,0.0,4.0,40.0,960.0,1
66,2022-02-18 19:39:32 +0000,Dumbbell Lunge,5,18.143716,0.0,0.0,0.0,4.0,40.0,800.0,1


# Weekly Workout Stats

#### My split:
- Back, Biceps, Shoulders (Day 1)
- Chest, Triceps (Day 2)
- Quads, Hamstrings, Calves, Glutes (Day 3)

#### My goals:
- Min: 10, Max: 20 reps per muscle gorup, per training week (2 splits)
- Min: 8, Max: 12 reps per set
- Qualitative (and untracked, currently): Achieve maximum hyperbolic stimulus through improved mind-muscle connection, proper form, and progressive overloading within the safezone of 3-0 RIR

#### Methods:

To achieve peak hypertrophic stimulus, each set will be taken between 3-0 RIR (reps in reserve), or close to, if not at, failure. Within these parameters, I will aim for an average rep range of 8-12 RPS (reps per set)

In [312]:
tw_pivot = this_week.groupby("Exercise").sum().drop(columns=["multiplier", "Weight(kg)", "Weight(lb)"])

# Will worry about cardio tracking later
tw_pivot.drop(columns=["Duration(s)", "Distance(m)", "Incline"], inplace=True)

tw_pivot[["Reps", "Sets", "Volume(lb)"]]

,Reps,Sets,Volume(lb)
Exercise,,,
Back Squat,40,5,4600.0
Bicycle Crunch,10,1,0.0
Cable Tricep Pushdown,46,5,1760.0
Dumbbell Bench Press,56,5,5040.0
Dumbbell Bicep Curl,36,4,2160.0
Dumbbell Fly,59,5,2360.0
Dumbbell Front Raise,40,4,1600.0
Dumbbell Kickbacks,60,5,1800.0
Dumbbell Lunge,11,2,1760.0


In [313]:
# Takes array of strings
def get_data(exercises):
    return tw_pivot.loc[exercises]

### Back, Shoulder, Biceps

In [314]:
back = get_data([
    "Dumbbell Row",
    "V-Bar Pulldown"
])
back

,Reps,Volume(lb),Sets
Exercise,,,
Dumbbell Row,48,4600.0,5
V-Bar Pulldown,50,5250.0,5


In [315]:
shoulders = get_data([
    "Dumbbell Front Raise",
    "Dumbbell Shoulder Press"
])
shoulders

,Reps,Volume(lb),Sets
Exercise,,,
Dumbbell Front Raise,40,1600.0,4
Dumbbell Shoulder Press,37,2220.0,4


In [316]:
biceps = get_data([
    "Dumbbell Bicep Curl",
    "Hammer Curls"
])
biceps

,Reps,Volume(lb),Sets
Exercise,,,
Dumbbell Bicep Curl,36,2160.0,4
Hammer Curls,25,1500.0,4


### Chest, Triceps

In [317]:
chest = get_data([
    "Dumbbell Bench Press",
    "Dumbbell Fly"
])
chest

,Reps,Volume(lb),Sets
Exercise,,,
Dumbbell Bench Press,56,5040.0,5
Dumbbell Fly,59,2360.0,5


In [318]:
triceps = get_data([
    "Cable Tricep Pushdown",
    "Dumbbell Kickbacks"
])
triceps

,Reps,Volume(lb),Sets
Exercise,,,
Cable Tricep Pushdown,46,1760.0,5
Dumbbell Kickbacks,60,1800.0,5


### Quads, Hamstrings, Calves, Glutes

In [319]:
quads = get_data([
    "Back Squat",
    "Dumbbell Lunge"
])
quads

,Reps,Volume(lb),Sets
Exercise,,,
Back Squat,40,4600.0,5
Dumbbell Lunge,11,1760.0,2


In [320]:
hamstrings = get_data([
    "Romanian Deadlift"
])
hamstrings

,Reps,Volume(lb),Sets
Exercise,,,
Romanian Deadlift,50,5750.0,5


In [321]:
glutes = get_data([
    "Single Leg Kickback"
])
glutes

,Reps,Volume(lb),Sets
Exercise,,,
Single Leg Kickback,50,0.0,5


# Weekly Training Reflections

TODO: 2022-02-27